In [1]:
import networkx as nx
import pandas as pd
import pickle
import math
from neo4j import GraphDatabase, Transaction

In [2]:
import os
os.getcwd()

'C:\\Users\\naumo\\PycharmProjects\\IFC_test\\notebooks'

In [6]:
df = pd.read_excel(
    "../a_new/solution.xls",
    sheet_name=0,
    header=None,
    names=["GESN", "name"]
    ).dropna()
df

,GESN,name
0,06-01-001,Устройство фундаментных плит железобетонных пл...
1,06-01-024,Устройство стен подвалов и подпорных стен желе...
2,06-01-041,Устройство перекрытий безбалочных толщиной до ...
3,06-01-031,Устройство железобетонных стен и перегородок в...
4,09-03-014,Монтаж связей и распорок из одиночных и парных...
5,09-03-015,Монтаж прогонов при шаге ферм до 12 м при высо...
6,09-03-012,Монтаж стропильных и подстропильных ферм на вы...


In [7]:
df_gesn_links = pd.read_excel(
    "../a_new/solution.xls",
    sheet_name=1,
    ).dropna()
df_gesn_links

,pred,flw
0,09-03-015,09-03-014
1,09-03-012,09-03-015
2,06-01-031,09-03-012


In [23]:
G = pickle.load(open('../src/KJ.pickle', 'rb'))

In [6]:
group_driver = GraphDatabase.driver(
    "bolt://localhost:7688",
    auth=("neo4j", "23109900")
)
group_driver.verify_connectivity()

In [7]:
# def add_node(tx: Transaction, id_, stor_id_, type_, name, coords,):
#     q_create = "CREATE (a:Element {id: $id, stor_id: $stor_id, type: $type, name: $name, coordinates: $crd})"
#     tx.run(q_create, id=str(id_), stor_id=stor_id_, type=type_, name=name, crd=coords)
def add_node(tx: Transaction, id_, stor_id_, props_):
    q = """
    CREATE (n:Element)
    SET n = $props
    SET n.id = $id
    SET n.stor_id = $stor_id
    """
    tx.run(q, id=str(id_), stor_id=str(stor_id_), props=props_)

def add_edge(tx: Transaction, n, m):
    q_edge = '''
    MATCH (a:Element) WHERE a.id = $id1
    MATCH (b:Element) WHERE b.id = $id2
    MERGE (a)-[r:CONTAINS]->(b)
    '''
    tx.run(q_edge, id1=str(n), id2=str(m))

def add_ifc_class(tx: Transaction, storey_id, class_id, class_name):
    q_class = '''
    MATCH (s:Element) WHERE s.id = $s_id
    MERGE (s)-[r:CONTAINS]->(:IfcClass {id: $id, name: $name})
    '''
    tx.run(q_class, s_id=str(storey_id), id=str(class_id), name=class_name)

def add_el_to_class(tx: Transaction, class_id, element_id):
    q_cls_el = '''
    MATCH (c:IfcClass) WHERE c.id = $cls_id
    MATCH (e:Element) WHERE e.id = $el_id
    MERGE (c)-[:CONSISTS_OF]->(e)
    '''
    tx.run(q_cls_el, cls_id=str(class_id), el_id=str(element_id))

def traverse(tx:Transaction, id1, id2):
    q_traverse = '''
    MATCH (a:Element) WHERE a.id = $id1
    MATCH (b:Element) WHERE b.id = $id2
    MERGE (a)-[:TRAVERSE]->(b)
    '''
    tx.run(q_traverse, id1=str(id1), id2=str(id2))

def add_class_rel(tx: Transaction, pred_name: str, flw_name: str):
    q_rel = '''
    MATCH (a:IfcClass) WHERE a.name = $name1
    MATCH (b:IfcClass) WHERE b.name = $name2
    MERGE (a)-[r:FOLLOWS]->(b)
    '''
    tx.run(q_rel, name1=pred_name, name2=flw_name)

In [145]:
classes = (
    'IfcWall',
    # 'IfcDoor',
    'IfcBuildingElementProxy',
    # 'IfcWindow',
    'IfcSlab',
    "IfcFlowTerminal",
    "IfcFurniture",
    'IfcCurtainWall',)
# Create group data
def add_class(tx: Transaction, class_name: str):
    q_class = '''
    MERGE (n:IfcClass {name: $name})
    '''
    tx.run(q_class, name=class_name)

with group_driver.session() as session:
    session.run('MATCH (n) DETACH DELETE n')
    for i in classes:
        session.execute_write(add_class, i)

    session.execute_write(add_class_rel, 'IfcBuildingElementProxy', 'IfcWall')
    session.execute_write(add_class_rel, 'IfcBuildingElementProxy', 'IfcSlab')
    session.execute_write(add_class_rel, 'IfcWall', 'IfcWindow')
    session.execute_write(add_class_rel, 'IfcWall', "IfcFlowTerminal")
    session.execute_write(add_class_rel, 'IfcWall', 'IfcCurtainWall')
    session.execute_write(add_class_rel, 'IfcWall', "IfcFurniture")
    session.execute_write(add_class_rel, 'IfcBuildingElementProxy', 'IfcDoor')
    session.execute_write(add_class_rel, 'IfcDoor', 'IfcWindow')

In [8]:
element_driver = GraphDatabase.driver(
    "bolt://localhost:7687",
    auth=("neo4j", "23109900")
)
element_driver.verify_connectivity()
# element_driver.verify_authentication()

In [25]:
build_id = [node for node, data in G.nodes(data=True) if data.get('is_a') == 'IfcBuilding'][0]
build_id

142

In [39]:
q = '''
MATCH (n) WHERE n.is_a = 'IfcBuildingStorey'
RETURN n.id AS id, n.Elevation As elevation'''
level_df = pd.DataFrame(element_driver.session().run(q).data())
level_df.head()

,id,elevation
0,175,-700.0
1,179,0.0
2,185,2700.0
3,191,3530.0
4,197,4000.0


In [40]:
level_df.sort_values(by=['elevation'], inplace=True, ignore_index=True)
level_df.head()

,id,elevation
0,163,-5000.0
1,169,-2070.0
2,175,-700.0
3,179,0.0
4,185,2700.0


In [44]:
level_df

,id,elevation
0,163,-5000.0
1,169,-2070.0
2,175,-700.0
3,179,0.0
4,185,2700.0
5,191,3530.0
6,197,4000.0
7,203,7000.0
8,209,10000.0
9,215,13000.0


In [45]:
from src.nx_to_neo4j_converter import NxToNeo4jConverter

neo4j_exp = NxToNeo4jConverter()
neo4j_exp.connect_storeys(317, 323)

In [43]:
with element_driver.session() as session:
    for ind, row in list(level_df.iterrows())[1:]:
        print(ind)
        # if ind != 0:
        #     session.execute_write(traverse, pred_id, row.id)
        # pred_id = row.id

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


In [27]:
for j in G.successors(build_id):
    contained_classes = set(G.nodes[i]['is_a'] for i in G.successors(j))
    # contained_classes = set(G.nodes[i]['is_a'] for i in list(filter(
    #             lambda el_id: G.nodes[el_id]["coordinates"],
    #             G.successors(j))
    #         ))
    print(j, contained_classes)

163 {'IfcBuildingElementProxy', 'IfcStair', 'IfcWall', 'IfcSlab'}
169 {'IfcWall'}
175 {'IfcWall', 'IfcSlab'}
179 {'IfcBuildingElementProxy', 'IfcStair', 'IfcWall', 'IfcSlab'}
185 {'IfcWall'}
191 {'IfcWall'}
197 {'IfcBuildingElementProxy', 'IfcStair', 'IfcWall', 'IfcSlab'}
203 {'IfcBuildingElementProxy', 'IfcStair', 'IfcWall', 'IfcSlab'}
209 {'IfcBuildingElementProxy', 'IfcStair', 'IfcWall', 'IfcSlab'}
215 {'IfcBuildingElementProxy', 'IfcStair', 'IfcWall', 'IfcSlab'}
221 {'IfcBuildingElementProxy', 'IfcStair', 'IfcWall', 'IfcSlab'}
227 {'IfcBuildingElementProxy', 'IfcStair', 'IfcWall', 'IfcSlab'}
233 {'IfcBuildingElementProxy', 'IfcStair', 'IfcWall', 'IfcSlab'}
239 {'IfcBuildingElementProxy', 'IfcStair', 'IfcWall', 'IfcSlab'}
245 {'IfcBuildingElementProxy', 'IfcStair', 'IfcWall', 'IfcSlab'}
251 {'IfcBuildingElementProxy', 'IfcStair', 'IfcWall', 'IfcSlab'}
257 {'IfcBuildingElementProxy', 'IfcStair', 'IfcWall', 'IfcSlab'}
263 {'IfcBuildingElementProxy', 'IfcStair', 'IfcWall', 'IfcSlab'}
2

In [12]:
contained_classes = set(G.nodes[i]['is_a'] for i in list(filter(
                lambda el_id: G.nodes[el_id]["coordinates"],
                G.successors(163))
            ))
print(contained_classes)

{'IfcBuildingElementProxy', 'IfcWall'}


In [15]:
set(G.nodes[i]['is_a'] for i in G.successors(169))

{'IfcBuildingElementProxy', 'IfcWall'}

In [154]:
with element_driver.session(database="neo4j") as session:
    session.run('MATCH (n) DETACH DELETE n')

    build_id = [node for node, data in G.nodes(data=True) if data.get('is_a') == 'IfcBuilding'][0]
    # print(build_id)
    session.execute_write(add_node, build_id, None, G.nodes[build_id])
                          # None,
                          # G.nodes[build_id].get('is_a', None),
                          # G.nodes[build_id].get('name', None),
                          # G.nodes[build_id].get("coordinates", 0),
                          # )

    k = 0
    for stor_id in G.successors(build_id):
        k += 1
        data = G.nodes[stor_id]
        if data.get('is_a') != 'IfcBuildingStorey':
            continue

        session.execute_write(add_node, stor_id, None, data)
                              # None, data['is_a'], data['name'], data['coordinates'])
        session.execute_write(add_edge, build_id, stor_id)


        contained_classes = set(G.nodes[i]['is_a'] for i in list(filter(
                lambda el_id: G.nodes[el_id]["coordinates"],
                G.successors(stor_id))
            ))
        print(contained_classes)
        cls_to_id = dict()
        for j, cls_name in enumerate(contained_classes):
            cls_to_id[cls_name] = str(stor_id) + '_' + str(j)
            session.execute_write(add_ifc_class, stor_id, cls_to_id[cls_name], cls_name)

        for cls in contained_classes:
            elements = list(filter(
                lambda el_id: G.nodes[el_id]["is_a"] == cls and G.nodes[el_id]["coordinates"],
                G.successors(stor_id))
            )

            # Для упорядочивания элементов одной группы
            start_point = (0, 0)
            def compute_angle(el_id):
                coordinates = G.nodes[el_id]["coordinates"]
                x, y, z = coordinates[0], coordinates[1], coordinates[2]
                angle = math.atan2(y - start_point[1], x - start_point[0])
                return z, angle

            for j, i in enumerate(sorted(elements, key=compute_angle)):
                s_data = G.nodes[i]
                if s_data["coordinates"]:
                    session.execute_write(add_node, i, stor_id, s_data)
                                          # s_data['is_a'], s_data['name'], s_data["coordinates"])
                    session.execute_write(add_el_to_class, cls_to_id[cls], i)
                    if j > 0:
                        session.execute_write(traverse, prev_id, i)
                    prev_id = i
                    if j == 4:
                        break

            q_rel = '''
            MATCH (a:IfcClass)-[r:FOLLOWS]->(b:IfcClass)
            RETURN a.name AS type1, b.name AS type2
            '''
            result = pd.DataFrame(group_driver.session().run(q_rel).data())
            result.apply(
                lambda row: connect_chains(stor_id, row.type1, row.type2),
                # lambda row: print(row.type1, row.type2),
                axis=1
            )

        if k == 4:
            break

163 IfcBuildingElementProxy 2349858 IfcWall 2161316
<class 'str'>
MATCH (a:Element) WHERE a.id = '2349858'
        MATCH (b:Element) WHERE b.id = '2161316'
        MERGE (a)-[r:TRAVERSE]->(b)
        
169 IfcBuildingElementProxy 2349337 IfcWall 719728
<class 'str'>
MATCH (a:Element) WHERE a.id = '2349337'
        MATCH (b:Element) WHERE b.id = '719728'
        MERGE (a)-[r:TRAVERSE]->(b)
        
175 IfcBuildingElementProxy 2316551 IfcWall 749004
<class 'str'>
MATCH (a:Element) WHERE a.id = '2316551'
        MATCH (b:Element) WHERE b.id = '749004'
        MERGE (a)-[r:TRAVERSE]->(b)
        
179 IfcBuildingElementProxy 3198133 IfcSlab 702965
<class 'str'>
MATCH (a:Element) WHERE a.id = '3198133'
        MATCH (b:Element) WHERE b.id = '702965'
        MERGE (a)-[r:TRAVERSE]->(b)
        
179 IfcBuildingElementProxy 3198133 IfcWall 764843
<class 'str'>
MATCH (a:Element) WHERE a.id = '3198133'
        MATCH (b:Element) WHERE b.id = '764843'
        MERGE (a)-[r:TRAVERSE]->(b)
        
179

In [153]:
def connect_chains(stor_id, type1: str, type2: str):
    q_get_last = f'''MATCH (s:Element {{is_a: '{type1}', stor_id: '{str(stor_id)}' }})
    WHERE NOT (s)-[]->(:Element {{is_a: '{type1}' }})
    RETURN s.id AS id
    LIMIT 1'''
    last_res = element_driver.session().run(q_get_last).data()
    q_get_first = f'''MATCH (s:Element {{is_a: '{type2}', stor_id: '{str(stor_id)}' }})
    WHERE NOT (:Element {{is_a: '{type2}' }})-[]->(s)
    RETURN s.id AS id
    LIMIT 1'''
    first_res = element_driver.session().run(q_get_first).data()

    if len(last_res) > 0 and len(first_res) > 0:
        q_rel = f'''MATCH (a:Element) WHERE a.id = '{last_res[0]['id']}'
        MATCH (b:Element) WHERE b.id = '{first_res[0]['id']}'
        MERGE (a)-[r:TRAVERSE]->(b)
        '''
        element_driver.session().run(q_rel)

In [ ]:
with group_driver.session() as session:
    q_rel = '''
    MATCH (a:IfcClass)-[r:FOLLOWS]->(b:IfcClass)
    RETURN a.name AS type1, b.name AS type2
    '''
    result = pd.DataFrame(session.run(q_rel).data())
    result.apply(
        lambda row: connect_chains(row.type1, row.type2),
        # lambda row: print(row.type1, row.type2),
        axis=1
    )
    print(result)

In [102]:
connect_chains(163, 'IfcBuildingElementProxy', 'IfcWall')